In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.autograd as atgd
import torch.optim as optim
import torchvision

import torchvision.transforms as transforms

batchsize = 16

transform = transforms.Compose(
    [
    transforms.Resize((32,32)),    
    transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
    ])

train_dataset = torchvision.datasets.MNIST(root = './data/mnist', 
                           train = True, 
                           transform = transform,
                           download = True) 
test_dataset = torchvision.datasets.MNIST(root = './data/mnist',
                           train = False,
                           transform = transform,
                           download = True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchsize,
                                          shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batchsize,
                                          shuffle=True, num_workers=2)

print("Data load OK!")


# # linear 
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(32*32, 10)
#     def forward(self, x):
#         x = x.view(-1, 32 * 32)
#         x= self.fc1(x)  
#         return x
# net = Net()



# # Network with only fc layers 
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(32*32, 16*16)
#         self.fc2 = nn.Linear(16*16, 8*8)
#         self.fc3 = nn.Linear(8*8, 10)

#     def forward(self, x):
#         x = x.view(-1, 32 * 32)
#         x = func.relu(self.fc1(x))
#         x = func.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x
    
# net = Net()

# # Network with only  conv layers 
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(32*32, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = x.view(-1, 32 * 32)
#         x = func.relu(self.fc1(x))
#         x = func.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x
    
# net = Net()


# Network with 2 Conv2, 3fc
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 5)
        self.conv2 = nn.Conv2d(8, 16, 3)
#         self.conv3 = nn.Conv2d(16, 24, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 6*6, 10)
#         self.fc2 = nn.Linear(100, 64)
#         self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
#         print("1",x.size())
        x = self.pool(func.relu(self.conv1(x)))
#         print("2",x.size())
        x = self.pool(func.relu(self.conv2(x)))
#         x = self.pool(func.relu(self.conv3(x)))
#         print("3",x.size())
        x = x.view(-1, 16 * 6*6)
#         x = func.relu(self.fc1(x))
        x = self.fc1(x)
#         x = func.relu(self.fc2(x))
#         x = self.fc3(x)
        return x

net = Net()
print(net)


# # Network with 2 Conv2, 3fc
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
# #         print("1",x.size())
#         x = self.pool(func.relu(self.conv1(x)))
# #         print("2",x.size())
#         x = self.pool(func.relu(self.conv2(x)))
# #         print("3",x.size())
#         x = x.view(-1, 16 * 5 * 5)
#         x = func.relu(self.fc1(x))
#         x = func.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# net = Net()
print(net)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(5): 
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()     
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 1000 == 999:    
            print("epoch number:%d, batch number:%5d, loss: %.3f" % (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print("Training OK!")


correct = 0
total = 0
# with torch.no_grad():
for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    total += batchsize
    correct += (predicted == labels).sum().item()

print('Test accuracy: %.3f %%' % (100 * correct / total))


Processing...
Done!
Data load OK!
Net(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=576, out_features=10, bias=True)
)
Net(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=576, out_features=10, bias=True)
)
epoch number:1, batch number: 1000, loss: 1.059
epoch number:1, batch number: 2000, loss: 0.316
epoch number:1, batch number: 3000, loss: 0.217
epoch number:2, batch number: 1000, loss: 0.136
epoch number:2, batch number: 2000, loss: 0.121
epoch number:2, batch number: 3000, loss: 0.121
epoch number:3, batch number: 1000, loss: 0.106
epoch number:3, batch number: 2000, loss: 0.094
epoch number:3, batch number: 3000, lo

In [3]:
m = nn.Linear(20, 30)
input = atgd.Variable(torch.randn(128, 20))
output = m(input)
print(output.size())


torch.Size([128, 30])


In [ ]:
# this is a test for commit